In [1]:
import logging
import pickle

import numpy as np

root_dir = r'/data/students/jeryl/RLOR/RLOR-ASAP/data/'


def load(filename, root_dir=root_dir):
    return pickle.load(open(root_dir + filename, "rb"))


file_catalog = {
    "test": {
        20: "vrp20_test_seed1234.pkl",
        50: "vrp50_test_seed1234.pkl",
        100: "vrp100_test_seed1234.pkl",
        200: "vrp200_test_seed1234.pkl",
    },
    "eval": {
        20: "vrp20_validation_seed4321.pkl",
        50: "vrp50_validation_seed4321.pkl",
        100: "vrp100_validation_seed4321.pkl",
        200: "vrp200_test_seed1234.pkl",
    },
}


def make_instance(args):
    depot, loc, demand, capacity, *args = args
    grid_size = 1
    if len(args) > 0:
        depot_types, customer_types, grid_size = args
    return {
        "loc": np.array(loc) / grid_size,
        "demand": np.array(demand) / capacity,
        "depot": np.array(depot) / grid_size,
    }


class lazyClass:
    data = {
        "test": {},
        "eval": {},
    }

    def __getitem__(self, index):
        partition, nodes, idx = index
        if not (partition in self.data) or not (nodes in self.data[partition]):
            logging.warning(
                f"Data sepecified by ({partition}, {nodes}) was not initialized. Attepmting to load it for the first time."
            )
            data = load(file_catalog[partition][nodes])
            self.data[partition][nodes] = [make_instance(instance) for instance in data]

        return self.data[partition][nodes][idx]


VRPDataset = lazyClass()


In [2]:
eval_partition = "test"
max_nodes = 200
eval_data_idx = 0
data = VRPDataset[eval_partition, max_nodes, eval_data_idx]

In [3]:
print(data)

{'loc': array([[0.55426939, 0.18097824],
       [0.37612516, 0.51885549],
       [0.05188429, 0.76719676],
       [0.04856034, 0.81969283],
       [0.57490996, 0.83490177],
       [0.84522348, 0.17869101],
       [0.91656136, 0.93916396],
       [0.06335357, 0.83067851],
       [0.70899005, 0.25709062],
       [0.32474056, 0.57330857],
       [0.84326141, 0.2934209 ],
       [0.01093914, 0.46258112],
       [0.35876234, 0.19400373],
       [0.72120151, 0.79555722],
       [0.05224472, 0.58440705],
       [0.7063009 , 0.23747678],
       [0.03043255, 0.99487261],
       [0.15485826, 0.45972601],
       [0.00185245, 0.45132732],
       [0.98785687, 0.25486374],
       [0.24768139, 0.77574726],
       [0.93648976, 0.98855371],
       [0.78791124, 0.78039909],
       [0.23771805, 0.26610314],
       [0.66858426, 0.04927965],
       [0.39937985, 0.18371885],
       [0.07361847, 0.02375452],
       [0.47454784, 0.21239601],
       [0.11860571, 0.03047033],
       [0.70622552, 0.76120366],
  

In [4]:
import os
import logging
import numpy as np
import pandas as pd

class LazyVRPDataset:
    """
    A lazy-loading class for VRP dataset instances.
    Loads data only when accessed.
    """
    data = {
        "train": {},  # Training data
        "test": {},   # Test (Evaluation) data
    }

    def __getitem__(self, index):
        """
        Fetches a VRP instance lazily.

        Args:
            index (tuple): (partition, nodes, idx)
                - partition: "train" or "test"
                - nodes: Number of nodes in the instance
                - idx: Index of the instance within the dataset

        Returns:
            dict: A VRP instance in `make_instance` format including vehicle count.
        """
        partition, nodes, idx = index

        # Check if data is already loaded
        if not (partition in self.data) or not (nodes in self.data[partition]):
            logging.warning(
                f"Data specified by ({partition}, {nodes}) was not initialized. Attempting to load it for the first time."
            )
            data = load_vrp_txt_files(file_catalog[partition][nodes])
            self.data[partition][nodes] = [make_instance(instance) for instance in data]

        return self.data[partition][nodes][idx]


def load_vrp_txt_data(file_path):
    """
    Loads a VRP instance from a .txt file and extracts vehicle & customer details.

    Args:
        file_path (str): Path to the .txt file.

    Returns:
        tuple: (depot, loc, demand, capacity, vehicle_number, ready_time, due_date, service_time)
    """
    with open(file_path, "r") as file:
        lines = [line.strip() for line in file.readlines() if line.strip()]

    # Parse vehicle information
    vehicle_info = {
        "number": int(lines[3].split()[0]),  # Extract number of vehicles
        "capacity": int(lines[3].split()[1])  # Extract vehicle capacity
    }

    # Locate the start of customer data
    customer_start_idx = lines.index("CUSTOMER") + 2

    # Parse customer information
    customer_data = []
    for line in lines[customer_start_idx:]:
        parts = line.split()
        if len(parts) == 7:
            customer_data.append({
                "CUST NO.": int(parts[0]),
                "XCOORD.": int(parts[1]),
                "YCOORD.": int(parts[2]),
                "DEMAND": int(parts[3]),
                "READY TIME": int(parts[4]),
                "DUE DATE": int(parts[5]),
                "SERVICE TIME": int(parts[6]),
            })

    # Convert to required format for make_instance()
    depot = (customer_data[0]["XCOORD."], customer_data[0]["YCOORD."])  # Depot as first customer
    loc = [(cust["XCOORD."], cust["YCOORD."]) for cust in customer_data[1:]]  # Customers only
    demand = [cust["DEMAND"] for cust in customer_data[1:]]  # Customers' demands
    ready_time = [cust["READY TIME"] for cust in customer_data[1:]]  # Customers' ready time
    due_date = [cust["DUE DATE"] for cust in customer_data[1:]]  # Customers' due date
    service_time = [cust["SERVICE TIME"] for cust in customer_data[1:]]  # Customers' service time

    capacity = vehicle_info["capacity"]
    vehicle_number = vehicle_info["number"]  # Extract number of vehicles

    return (depot, loc, demand, capacity, vehicle_number, ready_time, due_date, service_time)


def make_instance(args):
    """
    Converts raw VRP instance data into the required structured format.

    Args:
        args (tuple): (depot, loc, demand, capacity, vehicle_number, ready_time, due_date, service_time, *optional_args)

    Returns:
        dict: Formatted instance with normalized data, including time window constraints.
    """
    depot, loc, demand, capacity, vehicle_number, ready_time, due_date, service_time, *args = args
    grid_size = 1  # Default grid size
    if len(args) > 0:
        depot_types, customer_types, grid_size = args  # Handle optional args

    return {
        "loc": np.array(loc) / grid_size,       # Normalize customer locations
        "demand": np.array(demand) / capacity,  # Normalize demand values
        "depot": np.array(depot) / grid_size,   # Normalize depot location
        "vehicle_number": vehicle_number,       # Number of available vehicles
        "ready_time": np.array(ready_time),     # Ready time for each customer
        "due_date": np.array(due_date),         # Due date for each customer
        "service_time": np.array(service_time), # Service time for each customer
    }


def load_vrp_txt_files(folder_path):
    """
    Loads all VRP problem instances from a specified folder.

    Args:
        folder_path (str): Path to the directory containing VRP .txt files.

    Returns:
        List of VRP instances in raw format (before `make_instance`)
    """
    instances = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        if file_name.endswith(".TXT"):  # Match case-sensitive file extensions
            try:
                instance = load_vrp_txt_data(file_path)
                instances.append(instance)
            except Exception as e:
                print(f"Error loading {file_name}: {e}")

    return instances


# Define file catalog (assumes train/test distinction based on folders)
file_catalog = {
    "train": {
        200: "C:/Users/Jeryl Salas/Documents/Capstone2/RLOR-main/data2/train_200/",
    },
    "test": {
        200: "C:/Users/Jeryl Salas/Documents/Capstone2/RLOR-main/data2/test_200/",
    }
}


# Instantiate Lazy Dataset Loader
VRPDataset = LazyVRPDataset()


In [5]:
eval_partition = "test"
max_nodes = 200
eval_data_idx = 0
data = VRPDataset[eval_partition, max_nodes, eval_data_idx]

In [6]:
print(data)

{'loc': array([[ 10.,  86.],
       [ 16.,  62.],
       [ 84.,  36.],
       [ 79.,  72.],
       [ 16.,  39.],
       [ 61.,  54.],
       [ 96.,  26.],
       [ 16.,  92.],
       [ 85., 106.],
       [  6.,  25.],
       [ 37., 125.],
       [ 87., 108.],
       [ 18., 104.],
       [ 61.,  48.],
       [ 97.,  29.],
       [112.,  67.],
       [ 50., 106.],
       [ 99.,  28.],
       [ 15.,   9.],
       [ 19.,  37.],
       [ 90., 101.],
       [125.,  68.],
       [127., 108.],
       [ 90.,  44.],
       [  4.,  23.],
       [ 17.,  37.],
       [113.,  32.],
       [ 63.,  57.],
       [ 94., 104.],
       [ 58.,  95.],
       [137.,  96.],
       [127., 112.],
       [103., 119.],
       [ 20.,  69.],
       [  2.,  28.],
       [ 88.,  29.],
       [ 86., 102.],
       [126., 112.],
       [ 61.,  34.],
       [113.,  69.],
       [109.,  78.],
       [ 21.,  48.],
       [ 86.,  36.],
       [ 97.,  12.],
       [121., 112.],
       [ 19.,  39.],
       [ 66.,  40.],
     

In [7]:
import numpy as np

def generate_vrp_data_formatted(dataset_size, num_customers=50, num_vehicles=5):
    """
    Generate VRP instances following the required format.

    Parameters:
        dataset_size (int): Number of instances to generate.
        num_customers (int): Number of customers in each instance.
        num_vehicles (int): Number of vehicles available.

    Returns:
        List[dict]: List of dictionaries containing 'loc', 'demand', 'depot', 'capacity', 'service_time', and 'depot_time'.
    """
    vehicle_capacity = 40  # C^v

    # Generate depot locations (one per instance)
    depot_location = np.random.uniform(size=(dataset_size, 2))

    # Generate customer locations
    customer_locations = np.random.uniform(size=(dataset_size, num_customers, 2))

    # Generate demand values uniformly from U(1,10) and normalize by vehicle capacity
    demands = np.random.randint(1, 11, size=(dataset_size, num_customers)) / vehicle_capacity

    # Vehicle capacity is the same for all instances
    capacities = np.full((dataset_size, num_vehicles), vehicle_capacity)  # Fixed per instance

    # Generate service end-time constraints
    service_end_times = np.random.uniform(50, 10000, size=(dataset_size, num_customers))
    depot_service_end_time = np.full(dataset_size, 10000)  # Fixed for depot

    # Format the dataset
    dataset = []
    for i in range(dataset_size):
        instance = {
            "loc": np.vstack([depot_location[i], customer_locations[i]]),  # Stack depot + customers
            "demand": np.hstack([[0], demands[i]]),  # Depot has zero demand
            "depot": depot_location[i],  # Store depot separately
            "capacity": capacities[i],  # Store vehicle capacity per instance
            "due_time": service_end_times[i],  # Per-customer service end times
            "depot_time": depot_service_end_time[i]  # Depot's service time
        }
        dataset.append(instance)

    return dataset

# Example usage:
dataset = generate_vrp_data_formatted(10)  # Generate 10 instances
print(dataset[0])  # View one instance


{'loc': array([[0.88707089, 0.56296109],
       [0.99750458, 0.13573741],
       [0.57485915, 0.59664451],
       [0.28309632, 0.62658618],
       [0.25386112, 0.82060065],
       [0.15438826, 0.21188026],
       [0.69423693, 0.25041912],
       [0.71776086, 0.80487577],
       [0.90377692, 0.47681887],
       [0.45747206, 0.498758  ],
       [0.59323001, 0.30877954],
       [0.19334047, 0.82282095],
       [0.92693547, 0.4888103 ],
       [0.91979926, 0.11805629],
       [0.47322908, 0.77927227],
       [0.40860488, 0.22072672],
       [0.79684386, 0.45205469],
       [0.58346408, 0.95671156],
       [0.49320459, 0.22667107],
       [0.78657367, 0.88297857],
       [0.23615174, 0.94969103],
       [0.02972533, 0.90817359],
       [0.89356993, 0.50412222],
       [0.41747541, 0.19154558],
       [0.10274128, 0.03674108],
       [0.58044433, 0.85345544],
       [0.09310675, 0.78588578],
       [0.42053026, 0.78589563],
       [0.81131053, 0.53616376],
       [0.7768019 , 0.06744174],
  